# Phase 4: Grid Data Integration

## Objective
Overlay all data layers onto Geosquare Grid for comparative analysis.

## Data Layers
1. ✅ **Population** (from dasymetric mapping)
2. ⏳ **LULC** - Land Use Land Cover
3. ⏳ **Night Lights** - VIIRS 2020 vs 2025
4. ⏳ **BNPB Hazards** - 6 risk layers
5. ⏳ **RTRW Zoning** - Spatial planning
6. ⏳ **OSM POI** - Points of interest
7. ⏳ **OSM Roads** - Road network

## Output
- `output/grid_integration/grid_tangsel_integrated.csv`
- `output/grid_integration/grid_oku_integrated.csv`

In [1]:
# === CELL 1: IMPORTS ===
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from rasterio.sample import sample_gen
import warnings
warnings.filterwarnings('ignore')

# Define paths relative to notebook location
NOTEBOOK_DIR = Path.cwd() if '__file__' not in locals() else Path(__file__).parent
PROJECT_ROOT = NOTEBOOK_DIR.parent
PHASE1_DIR = PROJECT_ROOT / 'phase1_data_hunt'
PHASE2_DIR = PROJECT_ROOT / 'phase2_satellite'
PHASE3_DIR = PROJECT_ROOT / 'phase3_dasymetric'
OUTPUT_DIR = NOTEBOOK_DIR / 'outputs'

print("Phase 4: Grid Data Integration")
print("Overlaying all data layers to Geosquare Grid")
print("="*60)

Phase 4: Grid Data Integration
Overlaying all data layers to Geosquare Grid


In [2]:
# === CELL 2: LOAD BASE GRIDS (from dasymetric mapping) ===
print("Loading base grids with population...")

# Load dasymetric grids (already have population)
grid_tangsel = pd.read_csv(PHASE3_DIR / 'outputs' / 'pop_grid_tangsel.csv')
grid_oku = pd.read_csv(PHASE3_DIR / 'outputs' / 'pop_grid_oku.csv')

print(f"✓ Tangsel: {len(grid_tangsel):,} grids")
print(f"✓ OKU:     {len(grid_oku):,} grids")

# Show current columns
print(f"\nCurrent columns: {list(grid_tangsel.columns)}")

Loading base grids with population...
✓ Tangsel: 67,649 grids
✓ OKU:     1,529,054 grids

Current columns: ['grid_id', 'kelurahan', 'building_area_m2', 'estimated_pop', 'pop_density_km2', 'lat', 'lon']


In [3]:
# === CELL 3: HELPER FUNCTION - SAMPLE RASTER ===
def sample_raster_at_points(raster_path, lons, lats, band=1):
    """
    Sample raster values at given coordinates (lon, lat).
    Handles CRS reprojection and NoData values properly.
    """
    import rasterio
    from rasterio.warp import transform
    import numpy as np
    
    with rasterio.open(raster_path) as src:
        # Check if reprojection needed
        if src.crs != 'EPSG:4326':
            # Reproject coordinates from WGS84 to raster CRS
            xs, ys = transform('EPSG:4326', src.crs, lons, lats)
            coords = [(x, y) for x, y in zip(xs, ys)]
        else:
            coords = [(lon, lat) for lon, lat in zip(lons, lats)]
        
        # Sample using src.sample()
        sampled = src.sample(coords, indexes=band)
        
        # Extract values, handle NoData
        nodata = src.nodata
        values = []
        for val in sampled:
            v = val[0]
            # Check for various NoData representations
            if nodata is not None and (v == nodata or abs(v - nodata) < 1e-6):
                values.append(np.nan)
            elif v == -9999.0 or v < -1e10:  # Common NoData values
                values.append(np.nan)
            else:
                values.append(v)
        
        return np.array(values)

print("✓ Helper functions loaded")

✓ Helper functions loaded


In [4]:
# === CELL 2: LOAD BASE GRIDS (from dasymetric mapping) ===
print("Loading base grids with population...")

# Load dasymetric grids (already have population)
grid_tangsel = pd.read_csv(PHASE3_DIR / 'outputs' / 'pop_grid_tangsel.csv')
grid_oku = pd.read_csv(PHASE3_DIR / 'outputs' / 'pop_grid_oku.csv')

print(f"✓ Tangsel: {len(grid_tangsel):,} grids")
print(f"✓ OKU:     {len(grid_oku):,} grids")

# Show current columns
print(f"\nCurrent columns: {list(grid_tangsel.columns)}")

Loading base grids with population...
✓ Tangsel: 67,649 grids
✓ OKU:     1,529,054 grids

Current columns: ['grid_id', 'kelurahan', 'building_area_m2', 'estimated_pop', 'pop_density_km2', 'lat', 'lon']


In [5]:
# === CELL 4: ADD LULC DATA ===
print("Adding LULC data...")

# LULC class mapping
LULC_MAPPING = {
    0: 'No Data',
    1: 'Water',
    2: 'Trees',
    4: 'Flooded Vegetation',
    5: 'Crops',
    7: 'Built Area',
    8: 'Bare Ground',
    11: 'Rangeland',
    12: 'Oil Palm'
}

# Sample LULC at grid centroids
print("Sampling Tangsel LULC...")
lulc_tangsel = sample_raster_at_points(
    str(PHASE2_DIR / 'data' / 'lulc' / 'lulc_tangsel_2025.tif'),
    grid_tangsel['lon'],
    grid_tangsel['lat']
)
grid_tangsel['lulc_class'] = pd.Series(lulc_tangsel).fillna(0).astype(int)
grid_tangsel['lulc_name'] = grid_tangsel['lulc_class'].map(LULC_MAPPING)

print("Sampling OKU LULC...")
lulc_oku = sample_raster_at_points(
    str(PHASE2_DIR / 'data' / 'lulc' / 'lulc_oku_2025.tif'),
    grid_oku['lon'],
    grid_oku['lat']
)
grid_oku['lulc_class'] = pd.Series(lulc_oku).fillna(0).astype(int)
grid_oku['lulc_name'] = grid_oku['lulc_class'].map(LULC_MAPPING)

# Summary
print("\nTangsel LULC distribution:")
print(grid_tangsel['lulc_name'].value_counts())

print("\nOKU LULC distribution:")
print(grid_oku['lulc_name'].value_counts())

print("\n✓ LULC data added")

Adding LULC data...
Sampling Tangsel LULC...
Sampling OKU LULC...

Tangsel LULC distribution:
lulc_name
Built Area            50312
Crops                  5563
Trees                  4249
Rangeland              2905
Water                  2010
Bare Ground            1434
No Data                 784
Flooded Vegetation      392
Name: count, dtype: int64

OKU LULC distribution:
lulc_name
Trees                 865167
Rangeland             275727
Crops                 152160
Oil Palm              143463
Built Area             52868
Flooded Vegetation     16351
Water                  12767
No Data                 5545
Bare Ground             5006
Name: count, dtype: int64

✓ LULC data added


## Layer 1: LULC (Land Use Land Cover)

In [6]:
# === CELL 4: ADD LULC DATA ===
print("Adding LULC data...")

# LULC class mapping
LULC_MAPPING = {
    0: 'No Data',
    1: 'Water',
    2: 'Trees',
    4: 'Flooded Vegetation',
    5: 'Crops',
    7: 'Built Area',
    8: 'Bare Ground',
    11: 'Rangeland',
    12: 'Oil Palm'
}

# Sample LULC at grid centroids
print("Sampling Tangsel LULC...")
lulc_tangsel = sample_raster_at_points(
    str(PHASE2_DIR / 'data' / 'lulc' / 'lulc_tangsel_2025.tif'),
    grid_tangsel['lon'],
    grid_tangsel['lat']
)
grid_tangsel['lulc_class'] = pd.Series(lulc_tangsel).fillna(0).astype(int)
grid_tangsel['lulc_name'] = grid_tangsel['lulc_class'].map(LULC_MAPPING)

print("Sampling OKU LULC...")
lulc_oku = sample_raster_at_points(
    str(PHASE2_DIR / 'data' / 'lulc' / 'lulc_oku_2025.tif'),
    grid_oku['lon'],
    grid_oku['lat']
)
grid_oku['lulc_class'] = pd.Series(lulc_oku).fillna(0).astype(int)
grid_oku['lulc_name'] = grid_oku['lulc_class'].map(LULC_MAPPING)

# Summary
print("\nTangsel LULC distribution:")
print(grid_tangsel['lulc_name'].value_counts())

print("\nOKU LULC distribution:")
print(grid_oku['lulc_name'].value_counts())

print("\n✓ LULC data added")

Adding LULC data...
Sampling Tangsel LULC...
Sampling OKU LULC...

Tangsel LULC distribution:
lulc_name
Built Area            50312
Crops                  5563
Trees                  4249
Rangeland              2905
Water                  2010
Bare Ground            1434
No Data                 784
Flooded Vegetation      392
Name: count, dtype: int64

OKU LULC distribution:
lulc_name
Trees                 865167
Rangeland             275727
Crops                 152160
Oil Palm              143463
Built Area             52868
Flooded Vegetation     16351
Water                  12767
No Data                 5545
Bare Ground             5006
Name: count, dtype: int64

✓ LULC data added


## Layer 2: Night Lights (VIIRS)

In [7]:
# === CELL 5: ADD NIGHT LIGHTS DATA ===
print("Adding Night Lights data...")

# Tangsel
print("Sampling Tangsel Night Lights...")
nl_tangsel_2020 = sample_raster_at_points(
    str(PHASE2_DIR / 'data' / 'nightlights' / 'tangsel_nightlights_2020.tif'),
    grid_tangsel['lon'],
    grid_tangsel['lat']
)
nl_tangsel_2025 = sample_raster_at_points(
    str(PHASE2_DIR / 'data' / 'nightlights' / 'tangsel_nightlights_2025.tif'),
    grid_tangsel['lon'],
    grid_tangsel['lat']
)
grid_tangsel['nightlight_2020'] = nl_tangsel_2020
grid_tangsel['nightlight_2025'] = nl_tangsel_2025
grid_tangsel['nightlight_change'] = nl_tangsel_2025 - nl_tangsel_2020

# OKU
print("Sampling OKU Night Lights...")
nl_oku_2020 = sample_raster_at_points(
    str(PHASE2_DIR / 'data' / 'nightlights' / 'oku_nightlights_2020.tif'),
    grid_oku['lon'],
    grid_oku['lat']
)
nl_oku_2025 = sample_raster_at_points(
    str(PHASE2_DIR / 'data' / 'nightlights' / 'oku_nightlights_2025.tif'),
    grid_oku['lon'],
    grid_oku['lat']
)
grid_oku['nightlight_2020'] = nl_oku_2020
grid_oku['nightlight_2025'] = nl_oku_2025
grid_oku['nightlight_change'] = nl_oku_2025 - nl_oku_2020

# Summary
print("\nTangsel Night Lights stats:")
print(f"  2020: mean={grid_tangsel['nightlight_2020'].mean():.2f}, max={grid_tangsel['nightlight_2020'].max():.2f}")
print(f"  2025: mean={grid_tangsel['nightlight_2025'].mean():.2f}, max={grid_tangsel['nightlight_2025'].max():.2f}")
print(f"  Change: mean={grid_tangsel['nightlight_change'].mean():.2f}")

print("\nOKU Night Lights stats:")
print(f"  2020: mean={grid_oku['nightlight_2020'].mean():.2f}, max={grid_oku['nightlight_2020'].max():.2f}")
print(f"  2025: mean={grid_oku['nightlight_2025'].mean():.2f}, max={grid_oku['nightlight_2025'].max():.2f}")
print(f"  Change: mean={grid_oku['nightlight_change'].mean():.2f}")

print("\n✓ Night Lights data added")

Adding Night Lights data...
Sampling Tangsel Night Lights...
Sampling OKU Night Lights...

Tangsel Night Lights stats:
  2020: mean=17.22, max=35.09
  2025: mean=17.20, max=55.37
  Change: mean=-0.02

OKU Night Lights stats:
  2020: mean=0.41, max=82.47
  2025: mean=0.07, max=3.82
  Change: mean=-0.35

✓ Night Lights data added


## Layer 3: BNPB Hazards (6 layers)

In [8]:
# === CELL 6: ADD BNPB HAZARD DATA ===
print("Adding BNPB Hazard data (6 layers)...")

HAZARD_LAYERS = {
    'hazard_floods': PHASE1_DIR / 'bnpb' / 'inarisk' / 'inarisk_hazard_floods.tif',
    'hazard_drought': PHASE1_DIR / 'bnpb' / 'inarisk' / 'inarisk_hazard_drought.tif',
    'hazard_landslide': PHASE1_DIR / 'bnpb' / 'inarisk' / 'inarisk_hazard_landslide.tif',
    'hazard_earthquake': PHASE1_DIR / 'bnpb' / 'inarisk' / 'inarisk_hazard_earthquake.tif',
    'hazard_extreme_weather': PHASE1_DIR / 'bnpb' / 'inarisk' / 'inarisk_hazard_extreme_weather.tif',
    'hazard_fire': PHASE1_DIR / 'bnpb' / 'inarisk' / 'inarisk_hazard_land_forest_fire.tif'
}

# Sample all hazard layers
for hazard_name, hazard_path in HAZARD_LAYERS.items():
    print(f"  Sampling {hazard_name}...")
    
    # Tangsel
    hazard_tangsel = sample_raster_at_points(
        str(hazard_path),
        grid_tangsel['lon'],
        grid_tangsel['lat']
    )
    grid_tangsel[hazard_name] = hazard_tangsel
    
    # OKU
    hazard_oku = sample_raster_at_points(
        str(hazard_path),
        grid_oku['lon'],
        grid_oku['lat']
    )
    grid_oku[hazard_name] = hazard_oku

# Calculate composite hazard score (mean of all hazards)
hazard_cols = list(HAZARD_LAYERS.keys())
grid_tangsel['hazard_composite'] = grid_tangsel[hazard_cols].mean(axis=1)
grid_oku['hazard_composite'] = grid_oku[hazard_cols].mean(axis=1)

# Summary
print("\nTangsel Hazard Summary (mean risk):")
for col in hazard_cols:
    print(f"  {col}: {grid_tangsel[col].mean():.3f}")
print(f"  Composite: {grid_tangsel['hazard_composite'].mean():.3f}")

print("\nOKU Hazard Summary (mean risk):")
for col in hazard_cols:
    print(f"  {col}: {grid_oku[col].mean():.3f}")
print(f"  Composite: {grid_oku['hazard_composite'].mean():.3f}")

print("\n✓ BNPB Hazard data added")

Adding BNPB Hazard data (6 layers)...
  Sampling hazard_floods...
  Sampling hazard_drought...
  Sampling hazard_landslide...
  Sampling hazard_earthquake...
  Sampling hazard_extreme_weather...
  Sampling hazard_fire...

Tangsel Hazard Summary (mean risk):
  hazard_floods: 0.578
  hazard_drought: 0.512
  hazard_landslide: 0.000
  hazard_earthquake: 0.467
  hazard_extreme_weather: 0.895
  hazard_fire: 0.002
  Composite: 0.397

OKU Hazard Summary (mean risk):
  hazard_floods: 0.623
  hazard_drought: 0.394
  hazard_landslide: 0.171
  hazard_earthquake: 0.159
  hazard_extreme_weather: 0.328
  hazard_fire: 0.433
  Composite: 0.315

✓ BNPB Hazard data added


## Layer 4: RTRW Zoning

In [9]:
# === CELL 7: ADD RTRW ZONING DATA ===
print("Adding RTRW Zoning data...")

# Load RTRW polygons
rtrw_tangsel = gpd.read_file(PHASE1_DIR / 'rtrw' / 'RTRW_KOTA_TANGERANG_SELATAN.geojson')
rtrw_oku = gpd.read_file(PHASE1_DIR / 'rtrw' / 'RTRW_OGAN_KOMERING_ULU.geojson')

print(f"✓ Loaded Tangsel RTRW: {len(rtrw_tangsel)} zones")
print(f"✓ Loaded OKU RTRW: {len(rtrw_oku)} zones")

# Convert grids to GeoDataFrame for spatial join
grid_tangsel_gdf = gpd.GeoDataFrame(
    grid_tangsel,
    geometry=gpd.points_from_xy(grid_tangsel['lon'], grid_tangsel['lat']),
    crs='EPSG:4326'
)
grid_oku_gdf = gpd.GeoDataFrame(
    grid_oku,
    geometry=gpd.points_from_xy(grid_oku['lon'], grid_oku['lat']),
    crs='EPSG:4326'
)

# Spatial join (point in polygon)
print("Spatial joining Tangsel...")
joined_tangsel = gpd.sjoin(
    grid_tangsel_gdf,
    rtrw_tangsel[['geometry', 'namobj', 'rtrpkk']],  # adjust column names as needed
    how='left',
    predicate='within'
)
grid_tangsel['rtrw_zone'] = joined_tangsel['rtrpkk'].values
grid_tangsel['rtrw_name'] = joined_tangsel['namobj'].values

print("Spatial joining OKU...")
joined_oku = gpd.sjoin(
    grid_oku_gdf,
    rtrw_oku[['geometry', 'namobj', 'rtrpkk']],  # adjust column names as needed
    how='left',
    predicate='within'
)
grid_oku['rtrw_zone'] = joined_oku['rtrpkk'].values
grid_oku['rtrw_name'] = joined_oku['namobj'].values

# Summary
print("\nTangsel RTRW distribution:")
print(grid_tangsel['rtrw_zone'].value_counts().head(10))

print("\nOKU RTRW distribution:")
print(grid_oku['rtrw_zone'].value_counts().head(10))

print("\n✓ RTRW Zoning data added")

Adding RTRW Zoning data...
✓ Loaded Tangsel RTRW: 25 zones
✓ Loaded OKU RTRW: 21 zones
Spatial joining Tangsel...
Spatial joining OKU...

Tangsel RTRW distribution:
rtrw_zone
Permukiman Perkotaan\r\n                          40754
Perdagangan Dan Jasa                              16774
Kawasan Peruntukan Umum Dan Sosial Lainnya         1628
Kawasan Sungai                                     1107
Kawasan Peruntukan Industri                         926
Kawasan Perlindungan Setempat                       869
Kawasan Peruntukan Umum Dan Sosial Lainnya\r\n      781
Kawasan Bandara                                     537
Kawasan Permukiman                                  501
Kawasan Pariwisata                                  469
Name: count, dtype: int64

OKU RTRW distribution:
rtrw_zone
Kawasan Perkebunan               561368
Kawasan Hutan Lindung            261970
Hutan Produksi Terbatas          163115
Pertanian Pangan Lahan Kering    121609
Hutan Produksi Tetap             101059
Kawas

## Layer 5: OSM POI (Points of Interest)

In [10]:
# === CELL 8: ADD OSM POI DATA ===
print("Adding OSM POI data...")

# Load POI
poi_tangsel = gpd.read_file(PHASE1_DIR / 'osm' / 'osm_business_tangsel.geojson')
poi_oku = gpd.read_file(PHASE1_DIR / 'osm' / 'osm_business_oku.geojson')

print(f"✓ Loaded Tangsel POI: {len(poi_tangsel):,} points")
print(f"✓ Loaded OKU POI: {len(poi_oku):,} points")

# Create buffer around grid centroids (25m = half grid size)
# Grid size is 50m x 50m, so 25m buffer captures POI within grid
print("Buffering grid centroids (25m)...")
grid_tangsel_gdf_buffered = grid_tangsel_gdf.copy()
grid_tangsel_gdf_buffered['geometry'] = grid_tangsel_gdf.to_crs('EPSG:32748').buffer(25).to_crs('EPSG:4326')

grid_oku_gdf_buffered = grid_oku_gdf.copy()
grid_oku_gdf_buffered['geometry'] = grid_oku_gdf.to_crs('EPSG:32748').buffer(25).to_crs('EPSG:4326')

# Spatial join POI to buffered grids
print("Counting POI per grid (Tangsel)...")
poi_joined_tangsel = gpd.sjoin(
    poi_tangsel[['geometry']],
    grid_tangsel_gdf_buffered[['geometry']],
    how='inner',
    predicate='within'
)
poi_counts_tangsel = poi_joined_tangsel.groupby('index_right').size()
grid_tangsel['poi_count'] = grid_tangsel.index.map(poi_counts_tangsel).fillna(0).astype(int)

print("Counting POI per grid (OKU)...")
poi_joined_oku = gpd.sjoin(
    poi_oku[['geometry']],
    grid_oku_gdf_buffered[['geometry']],
    how='inner',
    predicate='within'
)
poi_counts_oku = poi_joined_oku.groupby('index_right').size()
grid_oku['poi_count'] = grid_oku.index.map(poi_counts_oku).fillna(0).astype(int)

# Summary
print(f"\nTangsel POI distribution:")
print(f"  Grids with POI: {(grid_tangsel['poi_count'] > 0).sum():,} ({(grid_tangsel['poi_count'] > 0).sum() / len(grid_tangsel) * 100:.1f}%)")
print(f"  Max POI per grid: {grid_tangsel['poi_count'].max()}")
print(f"  Mean POI per grid: {grid_tangsel['poi_count'].mean():.2f}")

print(f"\nOKU POI distribution:")
print(f"  Grids with POI: {(grid_oku['poi_count'] > 0).sum():,} ({(grid_oku['poi_count'] > 0).sum() / len(grid_oku) * 100:.1f}%)")
print(f"  Max POI per grid: {grid_oku['poi_count'].max()}")
print(f"  Mean POI per grid: {grid_oku['poi_count'].mean():.2f}")

print("\n✓ OSM POI data added")

Adding OSM POI data...
✓ Loaded Tangsel POI: 1,999 points
✓ Loaded OKU POI: 131 points
Buffering grid centroids (25m)...
Counting POI per grid (Tangsel)...
Counting POI per grid (OKU)...

Tangsel POI distribution:
  Grids with POI: 625 (0.9%)
  Max POI per grid: 4
  Mean POI per grid: 0.01

OKU POI distribution:
  Grids with POI: 76 (0.0%)
  Max POI per grid: 5
  Mean POI per grid: 0.00

✓ OSM POI data added


## Layer 6: OSM Roads

In [11]:
# === CELL 9: ADD OSM ROADS DATA ===
print("Adding OSM Roads data...")

# Load roads
roads_tangsel = gpd.read_file(PHASE1_DIR / 'osm' / 'osm_roads_tangsel.geojson')
roads_oku = gpd.read_file(PHASE1_DIR / 'osm' / 'osm_roads_oku.geojson')

print(f"✓ Loaded Tangsel Roads: {len(roads_tangsel):,} segments")
print(f"✓ Loaded OKU Roads: {len(roads_oku):,} segments")

# Reuse buffered grids from POI cell
# Calculate road length per grid (intersection)
print("Calculating road length per grid (Tangsel)...")
# Project to metric CRS for accurate length calculation
roads_tangsel_proj = roads_tangsel.to_crs('EPSG:32748')
grid_tangsel_buffered_proj = grid_tangsel_gdf_buffered.to_crs('EPSG:32748')

# Spatial join roads to grids
roads_in_grids_tangsel = gpd.sjoin(
    roads_tangsel_proj,
    grid_tangsel_buffered_proj[['geometry']],
    how='inner',
    predicate='intersects'
)

# Calculate length per grid
roads_in_grids_tangsel['length_m'] = roads_in_grids_tangsel.geometry.length
road_lengths_tangsel = roads_in_grids_tangsel.groupby('index_right')['length_m'].sum()
grid_tangsel['road_length_m'] = grid_tangsel.index.map(road_lengths_tangsel).fillna(0)

# Calculate density (km/km²)
# Grid area = 50m x 50m = 0.0025 km²
grid_area_km2 = 0.0025
grid_tangsel['road_density'] = (grid_tangsel['road_length_m'] / 1000) / grid_area_km2

print("Calculating road length per grid (OKU)...")
roads_oku_proj = roads_oku.to_crs('EPSG:32748')
grid_oku_buffered_proj = grid_oku_gdf_buffered.to_crs('EPSG:32748')

roads_in_grids_oku = gpd.sjoin(
    roads_oku_proj,
    grid_oku_buffered_proj[['geometry']],
    how='inner',
    predicate='intersects'
)

roads_in_grids_oku['length_m'] = roads_in_grids_oku.geometry.length
road_lengths_oku = roads_in_grids_oku.groupby('index_right')['length_m'].sum()
grid_oku['road_length_m'] = grid_oku.index.map(road_lengths_oku).fillna(0)
grid_oku['road_density'] = (grid_oku['road_length_m'] / 1000) / grid_area_km2

# Summary
print(f"\nTangsel Roads distribution:")
print(f"  Grids with roads: {(grid_tangsel['road_length_m'] > 0).sum():,} ({(grid_tangsel['road_length_m'] > 0).sum() / len(grid_tangsel) * 100:.1f}%)")
print(f"  Mean road length per grid: {grid_tangsel['road_length_m'].mean():.1f}m")
print(f"  Mean road density: {grid_tangsel['road_density'].mean():.1f} km/km²")

print(f"\nOKU Roads distribution:")
print(f"  Grids with roads: {(grid_oku['road_length_m'] > 0).sum():,} ({(grid_oku['road_length_m'] > 0).sum() / len(grid_oku) * 100:.1f}%)")
print(f"  Mean road length per grid: {grid_oku['road_length_m'].mean():.1f}m")
print(f"  Mean road density: {grid_oku['road_density'].mean():.1f} km/km²")

print("\n✓ OSM Roads data added")

Adding OSM Roads data...
✓ Loaded Tangsel Roads: 44,857 segments
✓ Loaded OKU Roads: 6,620 segments
Calculating road length per grid (Tangsel)...
Calculating road length per grid (OKU)...

Tangsel Roads distribution:
  Grids with roads: 50,728 (75.0%)
  Mean road length per grid: 341.5m
  Mean road density: 136.6 km/km²

OKU Roads distribution:
  Grids with roads: 47,896 (3.1%)
  Mean road length per grid: 104.7m
  Mean road density: 41.9 km/km²

✓ OSM Roads data added


## Export Integrated Grids

In [12]:
# === CELL 10: EXPORT INTEGRATED GRIDS ===
import os

OUTPUT_DIR.mkdir(exist_ok=True)

print("Exporting integrated grids...")
print()

# ========== CSV (tabular, no geometry) ==========
print("Exporting CSV (no geometry)...")
grid_tangsel.to_csv(OUTPUT_DIR / 'grid_tangsel_integrated.csv', index=False)
grid_oku.to_csv(OUTPUT_DIR / 'grid_oku_integrated.csv', index=False)
print(f"✓ grid_tangsel_integrated.csv ({len(grid_tangsel):,} grids)")
print(f"✓ grid_oku_integrated.csv ({len(grid_oku):,} grids)")

print("\n" + "="*70)
print("EXPORT COMPLETE")
print("="*70)
print()
print(f"Output directory: {OUTPUT_DIR}")
print()
print(f"Final columns ({len(grid_tangsel.columns)}):")
for i, col in enumerate(grid_tangsel.columns, 1):
    print(f"  {i:2d}. {col}")

Exporting integrated grids...

Exporting CSV (no geometry)...
✓ grid_tangsel_integrated.csv (67,649 grids)
✓ grid_oku_integrated.csv (1,529,054 grids)

EXPORT COMPLETE

Output directory: /home/leonk/Documents/geosquare/Geosquare Data Science Challenge/phase4_grid_integration/outputs

Final columns (24):
   1. grid_id
   2. kelurahan
   3. building_area_m2
   4. estimated_pop
   5. pop_density_km2
   6. lat
   7. lon
   8. lulc_class
   9. lulc_name
  10. nightlight_2020
  11. nightlight_2025
  12. nightlight_change
  13. hazard_floods
  14. hazard_drought
  15. hazard_landslide
  16. hazard_earthquake
  17. hazard_extreme_weather
  18. hazard_fire
  19. hazard_composite
  20. rtrw_zone
  21. rtrw_name
  22. poi_count
  23. road_length_m
  24. road_density


In [13]:
# === CELL 11: SUMMARY STATISTICS ===
print("="*70)
print("GRID DATA INTEGRATION - SUMMARY")
print("="*70)

print(f"\nTangsel: {len(grid_tangsel):,} grids")
print(f"OKU:     {len(grid_oku):,} grids")

print(f"\nData Layers Integrated:")
print(f"  ✓ Population (dasymetric)")
print(f"  ✓ LULC (8 classes)")
print(f"  ✓ Night Lights (2020 vs 2025)")
print(f"  ✓ BNPB Hazards (6 layers + composite)")
print(f"  ✓ RTRW Zoning")
print(f"  ✓ OSM POI")
print(f"  ✓ OSM Roads")

print(f"\nOutput Files:")
print(f"  - {OUTPUT_DIR}/grid_tangsel_integrated.csv")
print(f"  - {OUTPUT_DIR}/grid_oku_integrated.csv")

print("\n" + "="*70)
print("PHASE 4 COMPLETE")
print("="*70)
print("\nNext: Phase 5 - Investment Memo (2-page PDF)")

GRID DATA INTEGRATION - SUMMARY

Tangsel: 67,649 grids
OKU:     1,529,054 grids

Data Layers Integrated:
  ✓ Population (dasymetric)
  ✓ LULC (8 classes)
  ✓ Night Lights (2020 vs 2025)
  ✓ BNPB Hazards (6 layers + composite)
  ✓ RTRW Zoning
  ✓ OSM POI
  ✓ OSM Roads

Output Files:
  - /home/leonk/Documents/geosquare/Geosquare Data Science Challenge/phase4_grid_integration/outputs/grid_tangsel_integrated.csv
  - /home/leonk/Documents/geosquare/Geosquare Data Science Challenge/phase4_grid_integration/outputs/grid_oku_integrated.csv

PHASE 4 COMPLETE

Next: Phase 5 - Investment Memo (2-page PDF)
